In [1]:
#!/usr/bin/evn python

"""
CMSC733 Spring 2019: Classical and Deep Learning Approaches for
Geometric Computer Vision
Project1: MyAutoPano: Phase 1 Starter Code

Author(s): 
Chahat Deep Singh (chahat@terpmail.umd.edu) 
PhD Student in Computer Science,
University of Maryland, College Park

Nitin J. Sanket (nitinsan@terpmail.umd.edu)
PhD Candidate in Computer Science,
University of Maryland, College Park
"""

#Code starts here:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.feature import peak_local_max

In [2]:
def readImageSet(folder_name):
    print("Reading images from ", folder_name)
    images = []
    files = os.listdir(folder_name)	
    files = sorted(files)
    print("Found ", files)
    for file in files:
        image_path = folder_name + "/" + file
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)			
        else:
            print("Error in loading image ", image)

    return images


In [3]:
def displayImages(image_array, name):

    concat = image_array[0]
    print(concat.shape)
    for l in range(1,len(image_array)):
        concat = np.concatenate((concat, image_array[l]), axis = 1)
    cv2.imshow(name, concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [13]:
def detectCorners(images, choice):
    print("detecting corners ...")
    detected_corners = []
    cmaps = []
    corner_images = []
    for image in images:
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        gray_image = np.float32(gray_image)


        if(choice == 1):
            print("using Harris corner detection method.")
            corner_strength = cv2.cornerHarris(gray_image,2,3,0.04)
            corner_strength[corner_strength<0.01*corner_strength.max()] = 0
            detected_corner = np.where(corner_strength>0.001*corner_strength.max())
            detected_corners.append(detected_corner)
            cmaps.append(corner_strength)
            image[corner_strength > 0.001*corner_strength.max()]=[0,0,255]
            corner_images.append(image)
        else:
            print("using Shi-Tomashi corner detection method.")
            dst = cv2.goodFeaturesToTrack(gray_image, 250 ,0.01, 10)
            dst = np.int0(dst)
            detected_corners.append(dst)
            for c in dst:
                x,y = c.ravel()
                cv2.circle(image,(x,y),3,(0, 0, 255),-1) 
                          
            corner_images.append(image)
            cmap = np.zeros(gray_image.shape) #not sure what to do
            cmaps.append(cmap)

    return detected_corners, cmaps, corner_images

In [6]:
def getFeatureDescriptor(gray_image,x,y, patch_size=40):
    patch = gray_image[x-patch_size//2:x+patch_size//2, y-patch_size//2:y+patch_size//2] 
    # gaussian blur
    patch = cv2.GaussianBlur(patch,(3,3),0)
    # subsample to 20% size or 1/5th
    patch = cv2.resize(patch, None, fx=0.2, fy=0.2, interpolation = cv2.INTER_CUBIC)
    feature = patch.reshape(-1)
    feature = (feature-feature.mean())/ np.std(feature)

    return feature

In [14]:
def getPairs(image_1, image_2, all_corners_1, all_corners_2, patch_size = 40, alpha = 0.8):

    gray_image1 = cv2.cvtColor(image_1,cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image_2,cv2.COLOR_BGR2GRAY)

    width, height = image_1.shape[:2]
    print("width = ", width, ", height = ", height)
    features_1,features_2 = [], []
    corners_1, corners_2 = [],[]

    print("all corner 1 len = ", len(all_corners_1))
    print("all corner 2 len = ", len(all_corners_2))
    
    for corner in all_corners_1:
        x,y = corner.ravel()
        
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width):
            features_1.append(getFeatureDescriptor(gray_image1, y,x)) 
            corners_1.append([x,y])
        else:
            print("ignored x, y", x, y)

    for corner in all_corners_2:
        x,y = corner.ravel()
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width):
            features_2.append(getFeatureDescriptor(gray_image2, y,x)) 
            corners_2.append([x,y]) 

    matched_pairs, match_level = [], []
    for i, feat_1 in enumerate(features_1):
        ssd = []  
        for j, feat_2 in enumerate(features_2):
            ssd.append(np.sum((feat_1 - feat_2)**2))
        top_matche = np.argmin(ssd)
        #if ssd[top_matches[0]] / ssd[top_matches[1]] < alpha:   
            #matched_pairs.append([corners_1[i] , corners_2[top_matches[0]]])
        matched_pairs.append([corners_1[i] , corners_2[top_matche]]) 
    print("matched pairs num = ", len(matched_pairs))
    return np.array(matched_pairs)

In [8]:
def showMatches(image_1, image_2, matched_pairs, partition_width = 20):
    

    concat = np.concatenate((image_1, image_2), axis = 1)
    corners_1 = matched_pairs[:,0].copy()
    corners_2  = matched_pairs[:,1].copy()
    corners_2[:,0] += image_1.shape[1]

    for (x1,y1) , (x2,y2) in zip(corners_1, corners_2):
        cv2.line(concat, (x1,y1), (x2,y2), (0, 0, 255), 1)
    
      
    cv2.imshow('Feature_matches', concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [9]:
def testShowMatches(image_1, image_2, partition_width = 20):
    matched_pairs= []
    I = np.linspace(10, 100, 10)
    for i in I:
        x1 = i
        y1 = i
        corner1 = np.int0(np.array([x1, y1]))

        x2 = i
        y2 = i
        corner2 = np.int0(np.array([x2, y2]))
        matched_pairs.append([corner1 , corner2])

    matched_pairs = np.array(matched_pairs)
    showMatches(image_1, image_2, matched_pairs, partition_width = 20)


In [10]:
def AdaptiveNonMaximalSuppression(images, C_maps, N_best):

    anms_img = []
    anms_corners = []
    for i,image in enumerate(images):

        cmap = C_maps[i]
        local_maximas = peak_local_max(cmap, min_distance=10)
        n_strong = local_maximas.shape[0]
        
        r = [np.Infinity for i in range(n_strong)]
        x=np.zeros((n_strong,1))
        y=np.zeros((n_strong,1))
        eu_dist = 0

        for i in range(n_strong):
            for j in range(n_strong):
                x_j = local_maximas[j][0]
                y_j = local_maximas[j][1]

                x_i = local_maximas[i][0]
                y_i = local_maximas[i][1]

                if(cmap[x_j, y_j] > cmap[x_i, y_i]):
                    eu_dist = np.square(x_j - x_i) + np.square(y_j - y_i)
                if r[i] > eu_dist:
                    r[i] = eu_dist
                    x[i] = x_j
                    y[i] = y_j

        index = np.argsort(r)
        index = np.flip(index)
        index = index[0:N_best]
        x_best=np.zeros((N_best,1))
        y_best=np.zeros((N_best,1))

        for i in range(N_best):
            x_best[i] = np.int0(x[index[i]])
            y_best[i] = np.int0(y[index[i]]) 
            cv2.circle(image, (y_best[i], x_best[i]), 3, (0, 255, 0), -1)

        anms_corner = np.int0(np.concatenate((x_best, y_best), axis = 1))
        anms_corners.append(anms_corner)
        anms_img.append(image)
    return anms_corners, anms_img

In [15]:
def main():
    # Add any Command Line arguments here
    # Parser = argparse.ArgumentParser()
    # Parser.add_argument('--NumFeatures', default=100, help='Number of best features to extract from each image, Default:100')
    
    # Args = Parser.parse_args()
    # NumFeatures = Args.NumFeatures

    """
    Read a set of images for Panorama stitching
    """
    print("main")
    folder_name = "/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1"
    use_harris = False
    images = readImageSet(folder_name)
    displayImages(images, "image_set")

    """
    Corner Detection
    Save Corner detection output as corners.png
    1 = harris, 2 = shi-tomashi
    """
    choice = 2
    if use_harris:
        choice = 1

    detected_corners, cmaps, corner_images = detectCorners(images, choice)
    displayImages(corner_images, "corners")
    """
    Perform ANMS: Adaptive Non-Maximal Suppression
    Save ANMS output as anms.png
    """
    if (choice == 1):
        print("Applying ALMS.")
        detected_corners, anms_image = AdaptiveNonMaximalSuppression(images, cmaps, 100)
        displayImages(anms_image, "anms_output")
    else:
        print("goodFeaturesToTrack is already using ALMS.") #review

    """
    Feature Descriptors
    Save Feature Descriptor output as FD.png
    """
    """
    Feature Matching
    Save Feature Matching output as matching.png
    """
    matched_pairs = getPairs(images[0], images[1], detected_corners[0], detected_corners[1], patch_size = 40, alpha = 0.9 )
    showMatches(images[0],images[1],matched_pairs, partition_width = 20)
    """
    Refine: RANSAC, Estimate Homography
    """


    """
    Image Warping + Blending
    Save Panorama output as mypano.png
    """





In [16]:
if __name__ == '__main__':
    main()

ed corners = 364 139
detected corners = 340 404
detected corners = 559 110
detected corners = 585 276
detected corners = 284 318
detected corners = 425 406
detected corners = 552 52
detected corners = 592 202
detected corners = 550 34
detected corners = 591 117
detected corners = 60 289
detected corners = 530 228
detected corners = 308 288
detected corners = 397 403
detected corners = 594 140
detected corners = 148 383
detected corners = 394 314
detected corners = 368 378
detected corners = 367 314
detected corners = 431 166
detected corners = 24 230
detected corners = 124 234
detected corners = 224 339
detected corners = 45 282
detected corners = 588 95
detected corners = 116 435
detected corners = 210 310
detected corners = 582 108
detected corners = 593 129
detected corners = 549 22
detected corners = 205 407
detected corners = 585 73
detected corners = 561 4
detected corners = 594 7
detected corners = 229 435
detected corners = 596 152
detected corners = 586 183
detected corners = 